In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom

In [2]:
def encrypt(text, key):
    iv = urandom(16)  # Initialization Vector de 16 bytes
    cipher = Cipher(algorithms.AES(key), modes.OFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(text) + encryptor.finalize()
    return iv + ciphertext

def decrypt(ciphertext, key):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.OFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

# Ejemplo de uso
mensaje_original = b"Ejemplo de cifrado AES-256 en modo OFB"
clave_secreta = urandom(32)  # Clave de 256 bits (32 bytes)

# Cifrado
cifrado = encrypt(mensaje_original, clave_secreta)
print("Texto cifrado:", cifrado.hex())

# Descifrado
texto_descifrado = decrypt(cifrado, clave_secreta)
print("Texto descifrado:", texto_descifrado.decode('utf-8'))


Texto cifrado: b734a6871ce1c7641021ad9bf8b31c29096232eb088f3ac7f0d48c0f725237bf87b75aa3ffaa03f9715047d4fc4361bf1a3bedd00ad5
Texto descifrado: Ejemplo de cifrado AES-256 en modo OFB


In [3]:
import sqlite3
import pandas as pd

In [8]:
db_path = r'C:\Users\Pc\Documents\Talk_to_Eve_mvp\users\botChuck\botChuck.db'
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM sessions_history", conn)
conn.close()

In [11]:
df.head(2)

message_id  session_id  role  \
0           1           1  User   
1           2           1   Eve   

                                             message  depression_score  \
0  Ahoy there, Eve. I reckon I'm feeling a bit wo...          0.763857   
1  I'm sorry to hear that. It sounds like the cha...               NaN   

   stress_score  admiration  amusement     anger  annoyance  ...      love  \
0      0.763857    0.004892   0.001819  0.003801   0.056231  ...  0.002574   
1           NaN         NaN        NaN       NaN        NaN  ...       NaN   

   nervousness   neutral  optimism     pride  realization    relief   remorse  \
0      0.04389  0.078335   0.00784  0.000613     0.024919  0.003877  0.007623   
1          NaN       NaN       NaN       NaN          NaN       NaN       NaN   

   sadness  surprise  
0  0.43128   0.00189  
1      NaN       NaN  

[2 rows x 34 columns]

In [13]:
cifred_messages = []
for i in range(len(df)):
    cifred_messages.append(encrypt(df['message'][i].encode('utf-8'), clave_secreta))

In [16]:
cifred_messages[0]

b"\xc4\x8cF\x86qv\x15\x11\xbf\xed\x8a\xf7\xb7\x06X\xd7\x1f\x0f\x1d\xe6\x9695i\xfaJ\xac\x04\x0e\x8f\x89\xfcD\xe3\xbe\xc2\xf5cnA\xe0\x0e/\x05\x8e(\rY)hB\x0c\x96\x00\xa1\xfa%#\x8b9\xae\x0b0\xe0\x13\xea5\x05V\x98\xb77P~T\xd5,\xa91e\xb0`\xb3\x1e\xea\xc6\x02^[1WI\xc42q9cL\xa6\xfbq\x19@\xb9^\xb9ee62\x946\x10\x88]_\xee\xff\x06\x8feJ\x93\xb1o3\xb2\xce\xe2\x93\x1e\x1e\xd3\x0c\xf9(\xef\x9b|\xfa+W\x84\xcb\xa5\xc9\x8d7v\x8b\xb6\xfe\x95-:\xe7\x8e\x81\x07\xebvj;\x84\xca\xf1\xf3\xdf\xe1\xe4\xb8\x0e\xa3\x1f\xd02'\xa1(\xb0\x15\xf8\x80\x12\x1a\xa0\xa4~\xb4*[\x10\xb2nT$N\x97\xbb8i\x0c\x8cH[\xdd\xff\xf6\x95\x9a\x1f\xd7\xb4\x1d1\xe7\x0f\xf4v\xac\x87"

In [17]:
decrypt(cifred_messages[0], clave_secreta).decode('utf-8')

"Ahoy there, Eve. I reckon I'm feeling a bit worn down today, to be honest. It's just been one of those rough patches at sea, you know? The long hours and isolation are really starting to take a toll on me."

## Nicer

In [18]:
import sqlite3
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from os import urandom

def encrypt(text, key):
    iv = urandom(16)  # Initialization Vector de 16 bytes
    cipher = Cipher(algorithms.AES(key), modes.OFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(text) + encryptor.finalize()
    return iv + ciphertext

def decrypt(text, key):
    iv = text[:16]
    ciphertext = text[16:]
    cipher = Cipher(algorithms.AES(key), modes.OFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

def get_encrypted_columns(conn, table_name):
    cursor = conn.cursor()

    cursor.execute(f'PRAGMA table_info({table_name})')
    columns_info = cursor.fetchall()

    columns_to_encrypt = [column_info[1] for column_info in columns_info if column_info[2] == 'BLOB']

    conn.close()
    return columns_to_encrypt


def encrypt_dataframe(df, key, columns_to_encrypt):
    for column in columns_to_encrypt:
        df[column] = df[column].apply(lambda x: encrypt(str(x).encode('utf-8'), key))
    return df

def decrypt_dataframe(df, key, columns_to_decrypt):
    for column in columns_to_decrypt:
        df[column] = df[column].apply(lambda x: decrypt(x, key).decode('utf-8'))
    return df

# Obtén tu DataFrame df

# Nombre de tu tabla en la base de datos
table_name = 'datos'

# Clave de cifrado
key = urandom(32)  # Clave de 256 bits (32 bytes)



In [19]:
# test dataframe
data = {'name': ['John', 'Anna', 'Peter', 'Linda'],
        'age': [24, 13, 53, 33],
        'message': ['hello', 'hi', 'how are you?', 'bye']}
df = pd.DataFrame(data)

In [20]:
encrypted_df = encrypt_dataframe(df, key, ['message'])

In [21]:
decrypted_df = decrypt_dataframe(encrypted_df, key, ['message'])

In [22]:
decrypted_df

name  age       message
0   John   24         hello
1   Anna   13            hi
2  Peter   53  how are you?
3  Linda   33           bye

In [ ]:


# Ejemplo de uso
table_name = 'datos'
key = tu_clave_secreta

# Obtén datos de la base de datos utilizando pandas
conn = sqlite3.connect('tu_base_de_datos.db')
query = f'SELECT * FROM {table_name}'
df_from_db = pd.read_sql_query(query, conn)
conn.close()

# Obtén las columnas que deben ser descifradas
columns_to_decrypt = get_columns_to_decrypt(table_name)

# Descifra las columnas correspondientes en el DataFrame
df_decrypted = decrypt_dataframe(df_from_db.copy(), key, columns_to_decrypt)

# Ahora df_decrypted contiene los datos descifrados
print(df_decrypted)